In [ ]:
import json
import librosa
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd

import math
import librosa
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split
import re
from collections import Counter

import os
import json
import torch

import importlib.resources
import io

In [ ]:
KFOLD = 4
ROOT_ = '/Users/k.bykov/ml_assignment/data'
#'2.6.0'

In [ ]:
#%pip install torch==2.6.0

In [ ]:
#%pip install librosa

In [ ]:
#%pip install matplotlib-inline=="0.1.5"

In [ ]:
#%pip install mir_eval

In [ ]:
#%pip install pyrubberband

In [ ]:
#%pip install sortedcontainers

In [ ]:
#pip install git+https://github.com/enrupt/BTC_kosteecks_fork.git --force-reinstall

In [ ]:
from btc.utils import logger
from btc.btc_model import *
from btc.utils.mir_eval_modules import audio_file_to_features, idx2chord, idx2voca_chord
from btc.utils.hparams import *

In [ ]:
with open(ROOT_+'/data.json', 'r') as file:
    data = json.load(file)

def to_lab_fmt(json_struct):
    name = json_struct['path'].split("/")[2][:-4]
    chords = json_struct['chords']
    path = os.path.join(ROOT_, 'isophonic', name+'.lab')
    with open(path, 'w') as file:
        for c in chords:
            file.write(str(c['time']))
            file.write(' ')
            file.write(str(float(c['time']) + float(c['duration'])))
            file.write(' ')
            val = c['value']
            if val is None:
                val = 'N'
            file.write(val)
            file.write('\n')
for row in data:
    to_lab_fmt(row)

In [ ]:
def init_config():
    run_config_path = importlib.resources.files("btc").joinpath("run_config.yaml")
    content = run_config_path.read_text(encoding='ascii')
    config = HParams.load_str(content)
    config.feature['large_voca'] = True
    config.model['num_chords'] = 170
    return HParams.load_str(content)

def load_model_mean_std(config):
    model_path = importlib.resources.files("btc").joinpath("test/btc_model_large_voca.pt")
    model_content = model_path.read_bytes()
    buffer = io.BytesIO()
    buffer.write(model_content)
    buffer.seek(0)
    
    checkpoint = torch.load(buffer, map_location=torch.device('cpu'), weights_only=False)
    mean = checkpoint['mean']
    std = checkpoint['std']
    model = BTC_model(config=config.model)
    model.load_state_dict(checkpoint['model'])
    return model, mean, std

def gen_lab_lines(mp3_path, config):
    feature, feature_per_second, song_length_second = \
    audio_file_to_features(mp3_path, config)
    feature = feature.T
    feature = (feature - mean) / std
    time_unit = feature_per_second
    n_timestep = config.model['timestep']
    
    num_pad = n_timestep - (feature.shape[0] % n_timestep)
    feature = np.pad(feature, ((0, num_pad), (0, 0)), mode="constant", constant_values=0)
    num_instance = feature.shape[0] // n_timestep
    start_time = 0.0
    lines = []
    with torch.no_grad():
        model.eval()
        feature = torch.tensor(feature, dtype=torch.float32).unsqueeze(0).to(device)
        for t in range(num_instance):
            self_attn_output, _ = model.self_attn_layers(feature[:, n_timestep * t:n_timestep * (t + 1), :])
            prediction, _ = model.output_layer(self_attn_output)
            prediction = prediction.squeeze()
            for i in range(n_timestep):
                if t == 0 and i == 0:
                    prev_chord = prediction[i].item()
                    continue
                if prediction[i].item() != prev_chord:
                    lines.append(
                        '%.6f %.6f %s\n' % (
                            start_time, time_unit * (n_timestep * t + i), idx_to_chord[prev_chord]))
                    start_time = time_unit * (n_timestep * t + i)
                    prev_chord = prediction[i].item()
                if t == num_instance - 1 and i + num_pad == n_timestep:
                    if start_time != time_unit * (n_timestep * t + i):
                        lines.append(
                            '%.6f %.6f %s\n' % (
                                start_time, time_unit * (n_timestep * t + i), idx_to_chord[prev_chord]))
                    break
        return lines


logger.logging_verbosity(1)
device = "cpu"
config = init_config()
model, mean, std = load_model_mean_std(config)
idx_to_chord = idx2voca_chord()
lines = gen_lab_lines(ROOT_+'/isophonic/z9EaBjFvQpc.mp3', config)
print(lines)

In [ ]:
from btc.utils.chord_fix import *
find_closest('Am7')

In [ ]:
find_closest('Caug')

In [ ]:
find_closest('Em')

In [ ]:
from btc.utils.mir_eval_modules import large_voca_score_calculation
from btc.audio_dataset import AudioDataset
valid_dataset = AudioDataset(config, 
                             root_dir=ROOT_, 
                             dataset_names=('isophonic',),
                             num_workers = 8,
                             preprocessing=True, 
                             train=False, 
                             kfold=KFOLD,)
score_metrics = ['root', 'thirds', 'triads', 'sevenths', 'tetrads', 'majmin', 'mirex']
score_list_dict, song_length_list, average_score_dict = \
large_voca_score_calculation(valid_dataset=valid_dataset, 
                             config=config, 
                             model=model, 
                             model_type='btc', 
                             mean=mean, 
                             std=std, 
                             device=device)
for m in score_metrics:
    logger.info('==== %s score 1 is %.4f' % (m, average_score_dict[m]))

In [ ]:
valid_dataset.feature_string